### Imports

In [3]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper

import os

# Tiny Hacking Assignment!

Now that you have gotten a feeling for what value LangChain can bring you, it is time to get your hands dirty!

Your assignment is as follows: Think of a question you would like answered, and try to construct a ReAct agent to solve that question. 

Example questions:
- Trace the development of deep learning architectures from simple perceptrons to complex structures like GPT and BERT. What were the major breakthroughs and challenges at each stage?
- Analyze how data science techniques have been used to combat climate change. Include details on specific models and their applications in monitoring, predicting, and mitigating environmental impacts.

These questions would require various facts from different wikipedia pages, therewith making an agent an ideal way to answer such questions (if they are prompted correctly).

### Let's code!

In [ ]:
os.environ['OPENAI_API_KEY'] = '...'

In [2]:
# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')


# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

llm_with_tools = chat_llm.bind_tools([wikipedia_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        <Insert question:>

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

agent_executor.invoke({})